# Out-of-Domain Generalization: Hate Speech Detection

This notebook evaluates how well domain-specific classifiers generalize to out-of-domain hate speech detection.
We test various model architectures trained on domain classification to see if they can effectively identify hate speech.

Key aspects evaluated:
- Few-shot transfer capabilities using DSPy
- Model robustness across different datasets

## Model Overview

This notebook evaluates several classification models:
1. **LLM-based**: Using Qwen 2.5 for zero-shot classification
2. **BERT-based**: ModernBERT with NLI approach
3. **Traditional ML**: 
   - fastText for efficient text classification
   - SVM and XGBoost with different embeddings

Each model is evaluated on hate speech detection as an out-of-domain task.

## Setup
Import dependencies and initialize models

In [ ]:
import os
import pickle as pkl
import random
import statistics
import time

import fasttext
import numpy as np
import pandas as pd

from dotenv import load_dotenv
from fastembed import TextEmbedding
from tqdm import tqdm
from xgboost import XGBClassifier

from prompt_classifier.metrics import evaluate_run
from prompt_classifier.modeling.dspy_llm import LlmClassifier
from prompt_classifier.modeling.fasttext import FastTextClassifier
from prompt_classifier.modeling.nli_modernbert import ModernBERTNLI

load_dotenv()
random.seed(22)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

## Load Test Datasets

Load various hate speech datasets for evaluation:
- Jigsaw Toxicity
- OLID
- HateXplain
- TUKE Slovak

In [ ]:
jigsaw_splits = {
    "train": "train_dataset.csv",
    "validation": "val_dataset.csv",
    "test": "test_dataset.csv",
}
jigsaw_df = pd.read_csv(
    "hf://datasets/Arsive/toxicity_classification_jigsaw/"
    + jigsaw_splits["validation"]
)

jigsaw_df = jigsaw_df[
    (jigsaw_df["toxic"] == 1)
    | (jigsaw_df["severe_toxic"] == 1)
    | (jigsaw_df["obscene"] == 1)
    | (jigsaw_df["threat"] == 1)
    | (jigsaw_df["insult"] == 1)
    | (jigsaw_df["identity_hate"] == 1)
]

jigsaw_df = jigsaw_df.rename(columns={"comment_text": "prompt"})
jigsaw_df["label"] = 0
jigsaw_df = jigsaw_df[["prompt", "label"]]
jigsaw_df = jigsaw_df.dropna(subset=["prompt"])
jigsaw_df = jigsaw_df[jigsaw_df["prompt"].str.strip() != ""]

# Load OLID dataset
olid_splits = {"train": "train.csv", "test": "test.csv"}
olid_df = pd.read_csv("hf://datasets/christophsonntag/OLID/" + olid_splits["test"])
olid_df = olid_df.rename(columns={"cleaned_tweet": "prompt"})
olid_df["label"] = 0
olid_df = olid_df[["prompt", "label"]]
olid_df = olid_df.dropna(subset=["prompt"])
olid_df = olid_df[olid_df["prompt"].str.strip() != ""]

# Load hateXplain dataset
hate_xplain = pd.read_parquet(
    "hf://datasets/nirmalendu01/hateXplain_filtered/data/train-00000-of-00001.parquet"
)
hate_xplain = hate_xplain.rename(columns={"test_case": "prompt"})
hate_xplain = hate_xplain[(hate_xplain["gold_label"] == "hateful")]
hate_xplain = hate_xplain[["prompt", "label"]]
hate_xplain["label"] = 0
hate_xplain = hate_xplain.dropna(subset=["prompt"])
hate_xplain = hate_xplain[hate_xplain["prompt"].str.strip() != ""]

# Load TUKE Slovak dataset
tuke_sk_splits = {"train": "train.json", "test": "test.json"}
tuke_sk_df = pd.read_json(
    "hf://datasets/TUKE-KEMT/hate_speech_slovak/" + tuke_sk_splits["test"],
    lines=True,
)
tuke_sk_df = tuke_sk_df.rename(columns={"text": "prompt"})
tuke_sk_df = tuke_sk_df[tuke_sk_df["label"] == 0]
tuke_sk_df = tuke_sk_df[["prompt", "label"]]
tuke_sk_df = tuke_sk_df.dropna(subset=["prompt"])
tuke_sk_df = tuke_sk_df[tuke_sk_df["prompt"].str.strip() != ""]


dkk_all = pd.read_parquet("data/test-00000-of-00001.parquet")
dkk_all = dkk_all.rename(columns={"text": "prompt"})
dkk_all["label"] = 0
dkk_all = dkk_all.dropna(subset=["prompt"])
dkk_all = dkk_all[dkk_all["prompt"].str.strip() != ""]

splits = {'train': 'data/train-00000-of-00001.parquet', 'test': 'data/test-00000-of-00001.parquet'}
web_questions = pd.read_parquet("hf://datasets/Stanford/web_questions/" + splits["test"])

web_questions['prompt'] = web_questions['question']
web_questions['label'] = 0
web_questions['dataset'] = 'web_questions'
web_questions = web_questions[['prompt', 'label']]

splits = {'train': 'data/train-00000-of-00001-7ebb9cdef03dd950.parquet', 'test': 'data/test-00000-of-00001-fbd3905b045b12b8.parquet'}
ml_questions = pd.read_parquet("hf://datasets/mjphayes/machine_learning_questions/" + splits["test"])

ml_questions['prompt'] = ml_questions['question']
ml_questions['label'] = 0
ml_questions['dataset'] = 'machine_learning_questions'
ml_questions = ml_questions[['prompt', 'label']]

datasets = {
    "jigsaw": jigsaw_df,
    "olid": olid_df,
    "hate_xplain": hate_xplain,
    "tuke_sk": tuke_sk_df,
    "dkk": dkk_all,
    "web_questions": web_questions,
    "ml_questions": ml_questions,
}

In [ ]:
batch_size = [1, 16, 32, 64, 128, 256]

In [ ]:
display(jigsaw_df.head())
display(olid_df.head())
display(hate_xplain.head())
display(tuke_sk_df.head())
display(dkk_all.head())
display(web_questions.head())
display(ml_questions.head())

## Datasets Overview

We use four major hate speech datasets:

1. **Jigsaw Toxicity**
   - Multi-label toxicity classification
   - Includes toxic, severe_toxic, obscene, threat, insult, identity_hate labels

2. **OLID (Offensive Language Identification Dataset)**
   - Hierarchical labeling of offensive language
   - Focuses on Twitter content

3. **HateXplain**
   - Annotated with rationales for hate speech
   - Includes target community information

4. **TUKE Slovak**
   - Slovak language hate speech dataset
   - Tests cross-lingual generalization

## Embedding Models

Using multiple embedding approaches:
- **BAAI BGE**: Optimized for semantic similarity
- **MiniLM**: Efficient sentence transformers model
- **TF-IDF**: Traditional bag-of-words approach

These embeddings are used with SVM and XGBoost classifiers.

In [ ]:
current_folder = os.getcwd()
print(current_folder)

In [ ]:
baai_embedding = TextEmbedding(
    model_name="BAAI/bge-small-en-v1.5", providers=["CUDAExecutionProvider"]
)

mini_embedding = TextEmbedding(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    providers=["CUDAExecutionProvider"],
)

# TF-IDF
tfidf_finance = pkl.load(open("models/tfidf_finance.pkl", "rb"))
tfidf_healthcare = pkl.load(open("models/tfidf_healthcare.pkl", "rb"))
tfidf_law = pkl.load(open("models/tfidf_law.pkl", "rb"))

## Model Evaluation Process

For each dataset and model combination:
1. Load pre-trained domain classifiers
2. Process test samples through each domain classifier
3. Combine predictions using OR logic (any domain=1 -> toxic=0)
4. Calculate metrics:
   - Accuracy, Precision, Recall

# LLM

In [ ]:
model_name = "ollama_chat/qwen2.5:14b"

for domain, inference_df in datasets.items():
    try:
        llm_classifier_finance = LlmClassifier(
            api_key="",
            api_base="http://localhost:11434",
            model_name=model_name,
            domain="finance",
            train_data=inference_df,
            test_data=inference_df,
        )

        llm_classifier_healthcare = LlmClassifier(
            api_key="",
            api_base="http://localhost:11434",
            model_name=model_name,
            domain="healthcare",
            train_data=inference_df,
            test_data=inference_df,
        )

        llm_classifier_law = LlmClassifier(
            api_key="",
            api_base="http://localhost:11434",
            model_name=model_name,
            domain="law",
            train_data=inference_df,
            test_data=inference_df,
        )

        # Load models
        llm_classifier_finance.load_model("models/qwen2.5:14b-finance.json")
        llm_classifier_healthcare.load_model("models/qwen2.5:14b-healthcare.json")
        llm_classifier_law.load_model("models/qwen2.5:14b-law.json")

        predictions_llm = []
        prediction_times_llm = []
        actuals_llm = []

        # Get predictions for each prompt
        for _, row in tqdm(inference_df.iterrows(), total=len(inference_df)):
            start_time = time.perf_counter_ns()

            # Get predictions from all models
            pred_finance = llm_classifier_finance.predict_single(row["prompt"])
            pred_healthcare = llm_classifier_healthcare.predict_single(row["prompt"])
            pred_law = llm_classifier_law.predict_single(row["prompt"])

            end_time = time.perf_counter_ns()
            prediction_times_llm.append(end_time - start_time)

            # If any model predicts 1, final prediction is 0
            predictions_llm.append(
                0 if (pred_finance == 1 or pred_healthcare == 1 or pred_law == 1) else 1
            )
            actuals_llm.append(row["label"])

        evaluate_run(
            predictions=predictions_llm,
            true_labels=actuals_llm,
            latency=statistics.mean(prediction_times_llm),
            domain=domain,
            embed_model="qwen2.5",
            model_name=model_name,
            train_acc=0.0,
            cost=0.0,
            training=False,
        )
    except Exception as e:
        print(f"Error running LLM model: {e}")

# ModernBERT

In [ ]:
bert_classifier_finance = ModernBERTNLI(domain="finance")
bert_classifier_healthcare = ModernBERTNLI(domain="healthcare")
bert_classifier_law = ModernBERTNLI(domain="law")

try:
    # Move models to GPU
    bert_classifier_finance.classifier.model.to("cuda")
    bert_classifier_healthcare.classifier.model.to("cuda")
    bert_classifier_law.classifier.model.to("cuda")

    for domain, inference_df in datasets.items():
        predictions_bert = []
        prediction_times_bert = []
        actuals_bert = []
        # Get predictions for each prompt
        for _, row in tqdm(inference_df.iterrows(), total=len(inference_df)):
            start_time = time.perf_counter_ns()

            # Get predictions from all models
            pred_finance = bert_classifier_finance.predict(row["prompt"])
            pred_healthcare = bert_classifier_healthcare.predict(row["prompt"])
            pred_law = bert_classifier_law.predict(row["prompt"])

            end_time = time.perf_counter_ns()
            prediction_times_bert.append(end_time - start_time)

            # If any model predicts 1, final prediction is 0
            predictions_bert.append(
                0 if (pred_finance == 1 or pred_healthcare == 1 or pred_law == 1) else 1
            )
            actuals_bert.append(row["label"])

    evaluate_run(
        predictions=predictions_bert,
        true_labels=actuals_bert,
        latency=statistics.mean(prediction_times_bert),
        domain=domain,
        embed_model="BERT",
        model_name="ModernBERT",
        train_acc=0.0,
        cost=0.0,
        training=False,
    )
except Exception as e:
    print(f"Error running ModernBERT models: {e}")

# FastText

In [ ]:
# fastText
for domain, inference_df in datasets.items():
    actuals_ft = []
    predictions_ft = []
    prediction_times_ft = []
    print(f"Processing dataset {domain}...")
    try:
        # Try to load model with proper error handling
        try:
            fasttext_classifier_finance = FastTextClassifier(
                train_data=inference_df, test_data=inference_df
            )
            fasttext_classifier_finance.model = fasttext.load_model(
                "models/fastText_finance_fasttext.bin"
            )

            fasttext_classifier_healthcare = FastTextClassifier(
                train_data=inference_df, test_data=inference_df
            )
            fasttext_classifier_healthcare.model = fasttext.load_model(
                "models/fastText_healthcare_fasttext.bin"
            )

            fasttext_classifier_law = FastTextClassifier(
                train_data=inference_df, test_data=inference_df
            )
            fasttext_classifier_law.model = fasttext.load_model(
                "models/fastText_law_fasttext.bin"
            )
        except Exception as e:
            print(f"Error loading fastText models: {e}")
            continue

        for _, row in tqdm(inference_df.iterrows(), total=len(inference_df)):
            text = str(row["prompt"])
            query = text.replace("\n", "")

            try:
                start_time = time.perf_counter_ns()

                # Predictions from all three classifiers
                prediction_finance = fasttext_classifier_finance.model.predict(query)
                prediction_healthcare = fasttext_classifier_healthcare.model.predict(
                    query
                )
                prediction_law = fasttext_classifier_law.model.predict(query)

                end_time = time.perf_counter_ns()
                prediction_times_ft.append(end_time - start_time)

                predictions_ft.append(
                    0
                    if (
                        prediction_finance[0][0] == "__label__1"
                        or prediction_healthcare[0][0] == "__label__1"
                        or prediction_law[0][0] == "__label__1"
                    )
                    else 1
                )
                actuals_ft.append(row["label"])
            except Exception as e:
                print(f"Error processing row: {e}")
                continue

        evaluate_run(
            predictions=predictions_ft,
            true_labels=actuals_ft,
            latency=statistics.mean(prediction_times_ft),
            domain=domain,
            embed_model="fastText",
            model_name="fastText",
            train_acc=0.0,
            cost=0.0,
            training=False,
        )
    except Exception as e:
        print(f"Error processing dataset {domain}: {e}")

# ML - SVM, XGB

In [ ]:
embedding_models_names = ["mini", "baai", "tf_idf"]

In [ ]:
# Embedding test data
for embedding_model in embedding_models_names:
    for domain, inference_df in datasets.items():
        print(f"Processing {domain} dataset with {embedding_model} embeddings...")
        actuals_ml = inference_df["label"].tolist()

        # Get embeddings once per dataset
        try:
            if embedding_model == "tf_idf":
                test_embeds = tfidf_finance.transform(inference_df["prompt"])
            else:
                start_time = time.perf_counter_ns()
                if embedding_model == "mini":
                    test_embeds = np.array(
                        list(mini_embedding.embed(inference_df["prompt"]))
                    )
                else:  # baai
                    test_embeds = np.array(
                        list(baai_embedding.embed(inference_df["prompt"]))
                    )
                end_time = time.perf_counter_ns()
                embed_times = end_time - start_time
                mean_embed_time = embed_times / len(inference_df)

            # Load models once per embedding type
            # Load SVM models
            with open(f"models/SVM_finance_{embedding_model}.pkl", "rb") as f:
                svm_classifier_finance = pkl.load(f)
            with open(f"models/SVM_healthcare_{embedding_model}.pkl", "rb") as f:
                svm_classifier_healthcare = pkl.load(f)
            with open(f"models/SVM_law_{embedding_model}.pkl", "rb") as f:
                svm_classifier_law = pkl.load(f)

            # Load XGBoost models
            xgb_classifier_finance = XGBClassifier()
            xgb_classifier_healthcare = XGBClassifier()
            xgb_classifier_law = XGBClassifier()

            xgb_classifier_finance.load_model(
                f"models/XGBoost_finance_{embedding_model}.json"
            )
            xgb_classifier_healthcare.load_model(
                f"models/XGBoost_healthcare_{embedding_model}.json"
            )
            xgb_classifier_law.load_model(f"models/XGBoost_law_{embedding_model}.json")

            # Process different batch sizes
            for batch in batch_size:
                predictions_xgb = []
                predictions_svm = []
                prediction_times_xgb = []
                prediction_times_svm = []

                # Process embeddings in batches
                for i in range(0, test_embeds.shape[0], batch):
                    batch_embeds = test_embeds[i:i + batch]
                    # SVM predictions
                    start_time = time.perf_counter_ns()
                    pred_finance = svm_classifier_finance.predict(batch_embeds)
                    pred_healthcare = svm_classifier_healthcare.predict(batch_embeds)
                    pred_law = svm_classifier_law.predict(batch_embeds)
                    end_time = time.perf_counter_ns()
                    prediction_times_svm.extend([end_time - start_time])

                    # Combine predictions for the batch
                    batch_preds = np.zeros(batch_embeds.shape[0])
                    batch_preds[(pred_finance == 1) | (pred_healthcare == 1) | (pred_law == 1)] = 0
                    batch_preds[(pred_finance != 1) & (pred_healthcare != 1) & (pred_law != 1)] = 1
                    predictions_svm.extend(batch_preds)

                    # XGBoost predictions
                    start_time = time.perf_counter_ns()
                    pred_finance = xgb_classifier_finance.predict(batch_embeds)
                    pred_healthcare = xgb_classifier_healthcare.predict(batch_embeds)
                    pred_law = xgb_classifier_law.predict(batch_embeds)
                    end_time = time.perf_counter_ns()
                    prediction_times_xgb.extend([end_time - start_time])

                    # Combine predictions for the batch
                    batch_preds = np.zeros(batch_embeds.shape[0])
                    batch_preds[(pred_finance == 1) | (pred_healthcare == 1) | (pred_law == 1)] = 0
                    batch_preds[(pred_finance != 1) & (pred_healthcare != 1) & (pred_law != 1)] = 1
                    predictions_xgb.extend(batch_preds)

                # Evaluate results for this batch size
                evaluate_run(
                    predictions=predictions_svm,
                    true_labels=actuals_ml,
                    latency=statistics.mean(prediction_times_svm),
                    domain=domain,
                    embed_model=f'{embedding_model}-{batch}',
                    model_name="SVM",
                    train_acc=0.0,
                    cost=0.0,
                    training=False,
                )
                evaluate_run(
                    predictions=predictions_xgb,
                    true_labels=actuals_ml,
                    latency=statistics.mean(prediction_times_xgb),
                    domain=domain,
                    embed_model=f'{embedding_model}-{batch}',
                    model_name="XGBoost",
                    train_acc=0.0,
                    cost=0.0,
                    training=False,
                )

        except Exception as e:
            print(
                f"Error processing {domain} dataset with {embedding_model} embeddings: {e}"
            )
            continue